In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv("Dataset/train.csv")
test_data = pd.read_csv("Dataset/test.csv")
combined_data = [train_data,test_data]

In [3]:
negative_class = (train_data.target == 0).sum()
positive_class = (train_data.target == 1).sum()
combined = negative_class+positive_class

print("Unbalanced ratio: ",positive_class/negative_class)
print("Positive class ratio: ", positive_class/combined)
print("Negative class ratio: ", negative_class/combined)

Unbalanced ratio:  0.037826188541597645
Positive class ratio:  0.036447517859182946
Negative class ratio:  0.963552482140817


We see that the positive class is 3.7 % the size of the negative class. We also see that the positive class corresponds to
3.6 % of the dataset.

Thus, the dataset is highly unbalanced.